# Prototype: Alpha

| Properties      | Data    |
|---------------|-----------|
| *Labels* | `['BENIGN', 'DDoS']` |
| *Normalization* | `Min-Max` |
| *Sample Size* | `2000`|
| *Adversarial Attack* | `FGSM` |
| *Explanations* | `SHAP` |


---

In [98]:
# To import modules from the functions directory
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

## Data Preprocessing

In [99]:
import functions.data_preprocessing as dp
import importlib
importlib.reload(dp)

encoding_type = 0 # binary encoding
norm_type = 0 # min-max normalization
label_names = ['BENIGN', 'DDoS'] # labels to include
sample_size = 1000 # sample size for each label

label_df, feature_df = dp.preprocess(encoding_type, norm_type, label_names=label_names, sample_size=sample_size)
label_df.value_counts()

--- Combining all CICIDS2017 files ---
Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Friday-WorkingHours-Morning.pcap_ISCX.csv
Monday-WorkingHours.pcap_ISCX.csv
Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Tuesday-WorkingHours.pcap_ISCX.csv
Wednesday-workingHours.pcap_ISCX.csv
--- Removing NaN and Infinity values ---
Number of rows with NaN values:  1358
Removing NaN values....
Number of rows with Infinity values: 1509
Removing Infinity values....
--- Extracting labels ---
 Label
BENIGN    2271320
DDoS       128025
Name: count, dtype: int64
--- Sampling balanced data ---
Sample to shape: (2000, 79)
--- Splitting labels and features ---
--- Encoding labels as binary one-hot values ---
--- Removing irrelevant features ---
Removed Zero Columns: [' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' RST Flag Count', ' CWE Flag Count', ' ECE Flag Count', 'Fwd Avg B

BENIGN  ATTACK
False   True      1000
True    False     1000
Name: count, dtype: int64

## Split Data

In [100]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_df, label_df, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1600, 66) (400, 66) (1600, 2) (400, 2)


## Create IDS

In [101]:
import functions.intrusion_detection_system as ids
import importlib
importlib.reload(ids)

ids_model = ids.build_intrusion_detection_system(X_train, y_train, X_test, y_test)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5687 - loss: 0.6781 - val_accuracy: 0.7500 - val_loss: 0.6331
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7600 - loss: 0.6045 - val_accuracy: 0.8125 - val_loss: 0.5566
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8141 - loss: 0.5333 - val_accuracy: 0.8687 - val_loss: 0.4590
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8844 - loss: 0.4223 - val_accuracy: 0.9750 - val_loss: 0.3464
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9824 - loss: 0.2995 - val_accuracy: 0.9750 - val_loss: 0.2352
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9836 - loss: 0.1924 - val_accuracy: 0.9750 - val_loss: 0.1543
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9882 - loss: 0.1147 - val_accuracy: 0.9781 - val_loss: 0.1053
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9900 - loss: 0.0700 - val_accuracy: 0.9781 - val_loss

## Generate Attacks

In [102]:
import functions.attack_generator as ag
import importlib
importlib.reload(ag)

art_model = ag.convert_to_art_model(ids_model, X_train)

# import numpy as np
# target_label = np.zeros_like(y_train)
# target_label[:, 0] = 1 # desired predicted label = [1, 0] = BENIGN
# print(target_label[:3])

X_adv_fgsm = ag.generate_fgsm_attacks(art_model, X_train)
ag.evaluate_art_model(art_model, X_adv_fgsm, y_train)

Adversarial FGSM examples generated. Shape: (1600, 66)
Accuracy: 0.33375
              precision    recall  f1-score   support

      BENIGN       0.40      0.66      0.50       801
      ATTACK       0.01      0.00      0.00       799

   micro avg       0.33      0.33      0.33      1600
   macro avg       0.20      0.33      0.25      1600
weighted avg       0.20      0.33      0.25      1600
 samples avg       0.33      0.33      0.33      1600

Confusion Matrix: Positive == BENIGN
TN: 2, FP: 797, FN: 269, TP: 532


0.33375

## Explainer

In [103]:
import functions.explainer as exp
import importlib
importlib.reload(exp)

explainer = exp.generate_shap_explainer(ids_model, X_train)

shap_values = exp.generate_shap_values(explainer, X_train)
print(shap_values.shape)
shap_values_df = exp.convert_shap_values_to_pd(shap_values, X_train.columns)

shap_values_adv = exp.generate_shap_values(explainer, X_adv_fgsm)
print(shap_values_adv.shape)
shap_values_adv_df = exp.convert_shap_values_to_pd(shap_values_adv, X_train.columns)

PermutationExplainer explainer: 1601it [01:23, 16.94it/s]                          


(1600, 66)


PermutationExplainer explainer: 1601it [01:36, 14.78it/s]                          


(1600, 66)


## Detector

In [119]:
import functions.detector as det
import importlib
importlib.reload(det)
import numpy as np

# create dataframe
X, y = det.build_train_datasets(shap_values_df, shap_values_adv_df)
print(X.shape, y.shape)

# create normalizer
normalizer = det.create_min_max_normalizer(X)
# print(np.max(normalizer.data_max_))
# print(np.min(normalizer.data_min_))

# TODO: how to normalize? min/max should be consistent for all datasets
# normalize features
# X = normalizer.transform(X)
# print(np.max(X))
# print(np.min(X))

# split data
X_train_det, X_test_det, y_train_det, y_test_det = train_test_split(X, y, test_size=0.1, random_state=1503)
print(X_train_det.shape, X_test_det.shape, y_train_det.shape, y_test_det.shape)

# build detector
detector = det.build_detector(X_train_det, y_train_det, X_test_det, y_test_det)


# normalizer_adv = det.create_min_max_normalizer(shap_values_adv_df)
# print(np.max(normalizer_adv.data_max_))
# print(np.min(normalizer_adv.data_min_))

# shap_values_normalized = det.normalize_shap_values(shap_values_df)
# print(shap_values_normalized.shape)
# print(np.max(shap_values_normalized))
# print(np.min(shap_values_normalized))

(3200, 66) (3200, 2)
(2880, 66) (320, 66) (2880, 2) (320, 2)
Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.5321 - loss: 0.6907 - val_accuracy: 0.5243 - val_loss: 0.6821
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5359 - loss: 0.6794 - val_accuracy: 0.5538 - val_loss: 0.6628
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6102 - loss: 0.6540 - val_accuracy: 0.7465 - val_loss: 0.6225
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7335 - loss: 0.6189 - val_accuracy: 0.7865 - val_loss: 0.5727
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8154 - loss: 0.5698 - val_accuracy: 0.8663 - val_loss: 0.5207
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8592 - loss: 0.5329 - val_accuracy: 0.8663 - val_loss: 0.4610
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8894 - loss: 0.4596 - val_accuracy: 0.9444 - val_loss: 0.3948
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accu

---
## Test Process

In [120]:
print(X_test.shape)

(400, 66)


### Generate Attacks

In [121]:
X_test_adv = ag.generate_fgsm_attacks(art_model, X_test)

Adversarial FGSM examples generated. Shape: (400, 66)


### Generate Explanations

In [107]:
X_test_adv_shap_values = exp.generate_shap_values(explainer, X_test_adv)
X_test_shap_values = exp.generate_shap_values(explainer, X_test)
X_test_shap_values_df = exp.convert_shap_values_to_pd(X_test_shap_values, X_test.columns)
X_test_adv_shap_values_df = exp.convert_shap_values_to_pd(X_test_adv_shap_values, X_test.columns)

PermutationExplainer explainer: 401it [00:22,  9.81it/s]                         
PermutationExplainer explainer: 401it [00:23,  8.87it/s]                         


### Generate Datasets

In [122]:
X, y = det.build_train_datasets(X_test_shap_values_df, X_test_adv_shap_values_df)

### Preprocess Data

In [118]:
# print(np.max(X))
# print(np.min(X))
# X = normalizer.transform(X)
# print(np.max(X))
# print(np.min(X))

### Shuffle Data & Predict

In [123]:
from sklearn.utils import shuffle

X, y = shuffle(X, y, random_state=187)

y_pred = detector.predict(X)

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


### Evaluate

In [124]:
det.evaluate_model(y_pred, y)

Global Accuracy: 96.75%
              precision    recall  f1-score   support

      BENIGN       0.97      0.97      0.97       400
 ADVERSARIAL       0.97      0.97      0.97       400

    accuracy                           0.97       800
   macro avg       0.97      0.97      0.97       800
weighted avg       0.97      0.97      0.97       800

True Negative Rate: 96.75%
False Positive Rate: 3.25%
True Positive Rate: 96.75%
False Negative Rate: 3.25%
